In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD

In [41]:
class basicNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad = False)
        self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad = False)
        self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad = False)

        self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad = False)
        self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad = False)
        self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad = False)

        self.final_bias = nn.Parameter(torch.tensor(-16.0), requires_grad = False)

    def forward(self, input):
        input_to_top_relu = input *self.w00+self.b00
        top_relu_output = F.relu(input_to_top_relu)
        scaled_top_relu_output = top_relu_output*self.w01

        input_to_buttom_relu = input * self.w10+self.b10
        buttom_relu_output = F.relu(input_to_buttom_relu)
        scaled_buttom_relu_output = buttom_relu_output*self.w11

        input_to_final_relu = scaled_top_relu_output+scaled_buttom_relu_output+self.final_bias
        output = F.relu(input_to_final_relu)
        return output

    def random_function(self, inp):
        print("Random function op", inp)

In [42]:
input_data = torch.linspace(start =0, end = 1, steps=11)

model = basicNN()
output_values = model(input_data)
output_values

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0100, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])

In [43]:
input_data2 = torch.tensor([0. , 0.5, 1.])
model.forward(input_data2)

tensor([0.0000, 1.0100, 0.0000])

In [33]:
model.random_function("asdad")

Random function op asdad


In [34]:
model.forward(input_data)

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0100,  3.7320,  6.4540,
         9.1760, 11.8980, 14.6200])

In [35]:
model.forward(torch.tensor(1.0, requires_grad=False))

tensor(14.6200)

In [52]:
#Creating new network to update final_bias value by itself
class basicNN_train(nn.Module):
    def __init__(self):
        super().__init__()
        self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad = False)
        self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad = False)
        self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad = False)

        self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad = False)
        self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad = False)
        self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad = False)

        self.final_bias = nn.Parameter(torch.tensor(0.0), requires_grad = True)

    def forward(self, input):
        input_to_top_relu = input *self.w00+self.b00
        top_relu_output = F.relu(input_to_top_relu)
        scaled_top_relu_output = top_relu_output*self.w01

        input_to_buttom_relu = input * self.w10+self.b10
        buttom_relu_output = F.relu(input_to_buttom_relu)
        scaled_buttom_relu_output = buttom_relu_output*self.w11

        input_to_final_relu = scaled_top_relu_output+scaled_buttom_relu_output+self.final_bias
        output = F.relu(input_to_final_relu)
        return output


In [64]:
new_model = basicNN_train()
#optimizer 
optimizer = SGD(new_model.parameters(), lr =0.1)
epochs = 100

print('Final bias before optimization : ', new_model.final_bias.data)

inputs = torch.tensor([0. , 0.5, 1.])
labels = torch.tensor([0. , 1., 0.])

for epoch in range(epochs):
    total_loss = 0

    for iteration in range(len(inputs)):
        input_i = inputs[iteration]
        labels_i = labels[iteration]

        output_i = new_model(input_i)

        loss = (output_i -labels_i)**2

        loss.backward()

        total_loss += float(loss)
    
    if(total_loss <0.0001):
        print("Epochs : ", epoch)
        break

    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch Count: {epoch}, Loss: {total_loss}")

print(f"Final bias after optimization {new_model.final_bias.data}")

Final bias before optimization :  tensor(0.)
Epoch Count: 0, Loss: 256.3200988769531
Epoch Count: 1, Loss: 164.0448760986328
Epoch Count: 2, Loss: 104.98870849609375
Epoch Count: 3, Loss: 67.1927719116211
Epoch Count: 4, Loss: 43.00336837768555
Epoch Count: 5, Loss: 27.522159576416016
Epoch Count: 6, Loss: 17.614181518554688
Epoch Count: 7, Loss: 11.273076057434082
Epoch Count: 8, Loss: 7.21476936340332
Epoch Count: 9, Loss: 4.6174516677856445
Epoch Count: 10, Loss: 2.9551703929901123
Epoch Count: 11, Loss: 1.8913096189498901
Epoch Count: 12, Loss: 1.210437297821045
Epoch Count: 13, Loss: 0.7746805548667908
Epoch Count: 14, Loss: 0.49579527974128723
Epoch Count: 15, Loss: 0.3173092007637024
Epoch Count: 16, Loss: 0.20307788252830505
Epoch Count: 17, Loss: 0.12997011840343475
Epoch Count: 18, Loss: 0.0831809863448143
Epoch Count: 19, Loss: 0.053235918283462524
Epoch Count: 20, Loss: 0.03407112881541252
Epoch Count: 21, Loss: 0.021805522963404655
Epoch Count: 22, Loss: 0.0139555344358086

In [58]:
new_model(torch.tensor(0.5))

tensor(1.0081, grad_fn=<ReluBackward0>)

In [59]:
new_model1 = basicNN_train()
new_model1(torch.tensor(0.5))

tensor(17.0100, grad_fn=<ReluBackward0>)

In [ ]:
## deep inside
# new_model = basicNN_train()
# #optimizer 
# optimizer = SGD(new_model.parameters(), lr =0.1)
# epochs = 100

# print('Final bias before optimization : ', new_model.final_bias.data)

# inputs = torch.tensor([0. , 0.5, 1.])
# labels = torch.tensor([0. , 1., 0.])

# for epoch in range(epochs):
#     total_loss = 0

#     for iteration in range(len(inputs)):
#         input_i = inputs[iteration]
#         labels_i = labels[iteration]
#         print("input 1 =====>", input_i)
#         print("labels_i =====>", labels_i)
#         output_i = new_model(input_i)
#         print("output_i =====>",output_i)
#         loss = (output_i -labels_i)**2
#         print("loss wo sqare =====>",(output_i -labels_i))
#         print("loss =====>",loss)
#         loss.backward()

#         total_loss += float(loss)
    
#     if(total_loss <0.0001):
#         print("Epochs : ", epoch)
#         break

#     optimizer.step()
#     optimizer.zero_grad()
#     print(f"Epoch Count: {epoch}, Loss: {total_loss}")

# print(f"Final bias after optimization {new_model.final_bias.data}")